In [2]:
#1. existence assertions: Every crash occurred on highway

import pandas as pd

# Define file paths
crash_records_path = 'D:/Type_1_Data.csv'

# Load datasets
crash_df = pd.read_csv(crash_records_path)
participant_df = pd.read_csv(participant_records_path)
vehicle_df = pd.read_csv(vehicle_records_path)

# Validate every crash occurred on highway assuming 'Location_Type' holds this information
if 'Highway Number' in crash_df.columns:
    assert crash_df['Highway Number'].eq(26).all(), "Some crashes did not occur on highway 26."
    print("All crashes occured on highway 26.")
else:
    print("Highway Number column is missing.")

All crashes occured on highway 26.


In [3]:
#2. limit assertions: Recorded time of crashes must be valid times during the day 0 to 23.

import pandas as pd

# Load the data
data = pd.read_csv('D:/Type_1_Data.csv')

# Convert Crash Hour to integer
data['Crash Hour'] = data['Crash Hour'].astype(int)

# Find and print records where Crash Hour is not within the valid range
invalid_hours = data[~data['Crash Hour'].between(0, 23)]
if not invalid_hours.empty:
    print("Invalid crash hours found:")
    print(invalid_hours[['Crash ID', 'Crash Hour']])
else:
    print("All crash hours are valid.")

Invalid crash hours found:
     Crash ID  Crash Hour
2     1809637          99
302   1849683          99
471   1857364          99
484   1858806          99


In [6]:
#3.  intra-record assertions: If 'Alcohol-Involved Flag' is true, there must also be a corresponding police report in 'Investigating Agency.

import pandas as pd

# Load the dataset
file_path = 'D:\Type_1_Data.csv'  # Replace with the git path
data = pd.read_csv(file_path)

# Filter for records where 'Alcohol-Involved Flag' is 1 (Yes)
alcohol_involved = data[data['Alcohol-Involved Flag'] == 1]

# Agencies that indicate a police report has been received
report_received_agencies = [1, 2, 3, 4, 6, 7]

# Using assert to validate that all records with alcohol involvement have a proper police report
try:
    assert all(alcohol_involved['Investigating Agency'].isin(report_received_agencies))
    print("All alcohol-involved records have valid police reports.")
except AssertionError:
    invalid_reports = alcohol_involved[~alcohol_involved['Investigating Agency'].isin(report_received_agencies)]
    print(f"Found {len(invalid_reports)} alcohol-involved records without valid police reports:")
    print(invalid_reports[['Crash ID', 'Alcohol-Involved Flag', 'Investigating Agency']])


Found 41 alcohol-involved records without valid police reports:
     Crash ID  Alcohol-Involved Flag  Investigating Agency
4     1812266                    1.0                   NaN
46    1832302                    1.0                   NaN
61    1834407                    1.0                   NaN
115   1840093                    1.0                   NaN
123   1840583                    1.0                   NaN
133   1841013                    1.0                   NaN
142   1841361                    1.0                   NaN
151   1841693                    1.0                   NaN
152   1841708                    1.0                   NaN
155   1841806                    1.0                   NaN
167   1843196                    1.0                   NaN
201   1845326                    1.0                   NaN
203   1845474                    1.0                   NaN
205   1845619                    1.0                   NaN
206   1845672                    1.0               

In [7]:
#4 inter-record check assertions: 
#4.1 -Every 'Participant ID' in Vehicle Records should have a corresponding 'Crash ID' in Crash Records, confirming that every participant is linked to a valid crash event
#4.2 -All records associated with the same 'Crash ID' should be reported by the same investigating agency.

import pandas as pd

# Define file paths
crash_records_path = 'D:/Type_1_Data.csv'
participant_records_path = 'F:/quarter_3/data_engineering/git_class/dataeng_class/04_validate/DataType_2.csv'
vehicle_records_path = 'F:/quarter_3/data_engineering/git_class/dataeng_class/04_validate/DataType_3.csv'

# Load datasets
crash_df = pd.read_csv(crash_records_path)
participant_df = pd.read_csv(participant_records_path)
vehicle_df = pd.read_csv(vehicle_records_path)

# Inter-record assertions
# Merging participant records with vehicle records on 'Participant ID' and 'Vehicle ID'
merged_participant_vehicle = pd.merge(participant_df, vehicle_df, on='Participant ID', how='inner', suffixes=('_part', '_veh'))
merged_participant_crash = pd.merge(participant_df, crash_df, on='Crash ID', how='inner')

# Assertion 1: Every Participant ID in Vehicle_Records should have a corresponding Crash ID in Crash_Records
try:
    assert not merged_participant_crash['Crash ID'].isnull().any(), "Some Participant IDs do not link to a valid Crash ID"
    print("Assertion 1 Passed: Every 'Participant ID' in Vehicle Records should have a corresponding 'Crash ID' in Crash Records, confirming that every participant is linked to a valid crash event.")
except AssertionError as e:
    print(f"Assertion 1 Failed: {str(e)}")

# Assertion 2:
try:
    agency_check = crash_df.groupby('Crash ID')['Investigating Agency'].nunique().eq(1)
    assert agency_check.all(), "Different investigating agencies reported within the same Crash ID"
    print("Assertion 2 Passed: All records associated with the same 'Crash ID' should be reported by the same investigating agency unless officially transferred to another jurisdiction.")
except AssertionError as e:
    print(f"Assertion 2 Failed: {str(e)}")
    
print("All assertions processed.")




Assertion 1 Passed: Every 'Participant ID' in Vehicle Records should have a corresponding 'Crash ID' in Crash Records, confirming that every participant is linked to a valid crash event.
Assertion 2 Failed: Different investigating agencies reported within the same Crash ID
All assertions processed.


In [17]:
# Summary assertions
# 5.1 More than 5% of recorded crashes involve alcohol
# 5.2 The number of fatal crashes is more than 10% of total crashes
import pandas as pd

# Load the data
crash_records_path = 'D:/Type_1_Data.csv'
participant_records_path = 'F:/quarter_3/data_engineering/git_class/dataeng_class/04_validate/DataType_2.csv'
vehicle_records_path = 'F:/quarter_3/data_engineering/git_class/dataeng_class/04_validate/DataType_3.csv'

# More than 5% of recorded crashes involve alcohol
if 'Alcohol-Involved Flag' in data.columns:
    alcohol_crashes = data[data['Alcohol-Involved Flag'] == 1]
    if len(alcohol_crashes) < len(data) * 0.05:
        print("Less than 5% of the recorded crashes involve alcohol.")
    else:
        print("Assertion Passed: 5% or more of the recorded crashes involve alcohol.")

# The number of fatal crashes is more than 10% of total crashes
if 'Crash Severity' in data.columns:
    fatal_crashes = data[data['Crash Severity'] == 2]
    if len(fatal_crashes) < len(data) * 0.1:
        print("The number of fatal crashes is less than 10% of the total crashes reported.")
    else:
        print("Assertion Passed: The number of fatal crashes is 10% or more of the total crashes reported.")

Assertion Passed: 5% or more of the recorded crashes involve alcohol.
Assertion Passed: The number of fatal crashes is 10% or more of the total crashes reported.


In [11]:
#statistical distribution
#6.1 -Crashes are more frequent on weekdays compared to weekends.


import pandas as pd

# Load the dataset
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows you wish to display

file_path = 'D:\Type_1_Data.csv'  # Replace with the actual path to your dataset
data = pd.read_csv(file_path)

# Define day of week descriptions
day_of_week_descriptions = {
    1: 'Sunday',
    2: 'Monday',
    3: 'Tuesday',
    4: 'Wednesday',
    5: 'Thursday',
    6: 'Friday',
    7: 'Saturday'
}

# Filter records for weekends (Saturday and Sunday) and weekdays (Monday to Friday)
weekend_records = data[data['Week Day Code'].isin([1, 7])]
weekday_records = data[data['Week Day Code'].isin([2, 3, 4, 5, 6])]

# Count the number of crashes for each category
weekend_crash_count = len(weekend_records)
weekday_crash_count = len(weekday_records)

# Output results
print("Statistical Distribution of Crashes:")
print(f"Weekend Crashes ({len(weekend_records)} records):")
print(weekend_records[['Crash ID', 'Week Day Code']])
print("\nWeekday Crashes ({len(weekday_records)} records):")
print(weekday_records[['Crash ID', 'Week Day Code']])
print("\nWeekend Crash Count:", weekend_crash_count)
print("Weekday Crash Count:", weekday_crash_count)

# Compare crash frequency
if weekend_crash_count > weekday_crash_count:
    print("\nCrashes are more frequent on weekends compared to weekdays.")
elif weekend_crash_count < weekday_crash_count:
    print("\nCrashes are more frequent on weekdays compared to weekends.")
else:
    print("\nCrash frequency is the same on weekends and weekdays.")


Statistical Distribution of Crashes:
Weekend Crashes (143 records):
     Crash ID  Week Day Code
1     1809229            1.0
3     1810874            1.0
6     1816804            7.0
10    1826513            7.0
19    1828287            7.0
20    1828314            7.0
21    1828373            1.0
22    1828552            7.0
28    1829755            7.0
29    1829763            1.0
30    1829797            7.0
31    1829853            1.0
34    1829928            7.0
35    1829983            7.0
38    1830038            7.0
39    1830046            1.0
40    1830165            7.0
42    1830243            7.0
46    1832302            7.0
47    1832348            7.0
57    1834003            7.0
65    1834846            1.0
68    1835061            1.0
71    1835205            7.0
76    1835315            7.0
78    1835322            7.0
83    1835386            7.0
86    1835511            1.0
90    1836755            1.0
91    1836788            1.0
94    1837049            7.0
99  

In [24]:
#6.2 Crashes are lesser during rush hours.
import pandas as pd

# Load the data
data = pd.read_csv('D:\Type_1_Data.csv')


# Check if 'Crash Hour' column exists
if 'Crash Hour' in data.columns:
    # Define rush hours in the morning and evening
    morning_rush_hours = range(7, 10)  # From 7:00 AM to 9:59 AM
    evening_rush_hours = range(16, 19)  # From 4:00 PM to 6:59 PM
    
    # Filter data to find crashes during rush hours and non-rush hours
    rush_hour_crashes = data[data['Crash Hour'].isin(morning_rush_hours) | data['Crash Hour'].isin(evening_rush_hours)]
    non_rush_hour_crashes = data[~data['Crash Hour'].isin(morning_rush_hours) & ~data['Crash Hour'].isin(evening_rush_hours)]
    
    # Calculate the proportion of crashes during rush hours and non-rush hours
    total_crashes = len(data)
    rush_hour_proportion = len(rush_hour_crashes) / total_crashes
    non_rush_hour_proportion = len(non_rush_hour_crashes) / total_crashes

    # Print results
    print("Proportion of crashes during rush hours: {:.2%}".format(rush_hour_proportion))
    print("Proportion of crashes during non-rush hours: {:.2%}".format(non_rush_hour_proportion))

    # Assertion to check if rush hour crashes are significantly higher
    if rush_hour_proportion > non_rush_hour_proportion:
        print("Assertion Failed: A higher proportion of crashes occur during rush hours.")
    else:
        print("Assertion Passed: Crashes are lesser during rush hours.")
else:
    print("The 'Crash Hour' column is missing from the dataset. Please check the column names.")

Proportion of crashes during rush hours: 37.20%
Proportion of crashes during non-rush hours: 62.80%
Assertion Passed: Crashes are lesser during rush hours.
